In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Use BigQuery DataFrames to cluster and characterize complaints

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

The goal of this notebook is to demonstrate a comment characterization algorithm for an online business. We will accomplish this using [Google's PaLM 2](https://ai.google/discover/palm2/) and [KMeans clustering](https://en.wikipedia.org/wiki/K-means_clustering) in three steps:

1. Use PaLM2TextEmbeddingGenerator to [generate text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) for each of 10000 complaints sent to an online bank. If you're not familiar with what a text embedding is, it's a list of numbers that are like coordinates in an imaginary "meaning space" for sentences. (It's like [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), but for more general text.) The important point for our purposes is that similar sentences are close to each other in this imaginary space.
2. Use KMeans clustering to group together complaints whose text embeddings are near to eachother. This will give us sets of similar complaints, but we don't yet know _why_ these complaints are similar.
3. Prompt GeminiTextGenerator in English asking what the difference is between the groups of complaints that we got. Thanks to the power of modern LLMs, the response might give us a very good idea of what these complaints are all about, but remember to ["understand the limits of your dataset and model."](https://ai.google/responsibility/responsible-ai-practices/#:~:text=Understand%20the%20limitations%20of%20your%20dataset%20and%20model)

We will tie these pieces together in Python using BigQuery DataFrames. [Click here](https://cloud.google.com/bigquery/docs/dataframes-quickstart) to learn more about BigQuery DataFrames!

### Dataset

This notebook uses the [CFPB Consumer Complaint Database](https://console.cloud.google.com/marketplace/product/cfpb/complaint-database).

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (compute)
* BigQuery ML
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models), [Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,aiplatform.googleapis.com) to enable the following APIs:

  * BigQuery API
  * BigQuery Connection API
  * Vertex AI API

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
# set your project ID below
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id in gcloud
#! gcloud config set project {PROJECT_ID}

#### Authenticate your Google Cloud account

Depending on your Jupyter environment, you might have to manually authenticate. Follow the relevant instructions below.

**Vertex AI Workbench**

Do nothing, you are already authenticated.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [3]:
# ! gcloud auth login

**Colab**

Uncomment and run the following cell:

In [4]:
# from google.colab import auth
# auth.authenticate_user()

Now we are ready to use BigQuery DataFrames!

## Step 1: Text embedding 

BigQuery DataFrames setup

In [5]:
import bigframes.pandas as bf

# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bf.options.bigquery.project = PROJECT_ID

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bf.close_session()`. After that, you can reuse `bf.options.bigquery.location` to specify another location.

Data Input - read the data from a publicly available BigQuery dataset

In [6]:
input_df = bf.read_gbq("bigquery-public-data.cfpb_complaints.complaint_database")

In [7]:
issues_df = input_df[["consumer_complaint_narrative"]].dropna()
issues_df.peek(n=5) # View an arbitrary five complaints

,consumer_complaint_narrative
1053364,My Macy 's American Express account was taken ...
1053757,I am a victim of identity theft. The informati...
1053784,"In XXXX 2016, Amex took out $ XXXX.+ unauthori..."
1054237,I am not for sure the exact date of my loan it...
1054244,I entered a consumer credit transaction with t...


Downsample DataFrame to 10,000 records for model training.

In [8]:
# Choose 10,000 complaints randomly and store them in a column in a DataFrame
downsampled_issues_df = issues_df.sample(n=10000)

Generate the text embeddings

In [9]:
from bigframes.ml.llm import PaLM2TextEmbeddingGenerator

model = PaLM2TextEmbeddingGenerator() # No connection id needed

In [10]:
# Will take ~3 minutes to compute the embeddings
predicted_embeddings = model.predict(downsampled_issues_df)
# Notice the lists of numbers that are our text embeddings for each complaint
predicted_embeddings

,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,content
357,[ 1.33585772e-02 -3.76597494e-02 -6.14452176e-...,"{""token_count"":306,""truncated"":false}",,I decided to try XXXX services for my wife and...
428,[ 3.10036819e-04 -3.82593311e-02 -3.41922641e-...,"{""token_count"":134,""truncated"":false}",,XXXX I went to the bank in question ( XXXX XXX...
1319,[ 1.97481886e-02 -1.97448786e-02 -5.13443351e-...,"{""token_count"":215,""truncated"":false}",,I currently have a home loan with my ex husban...
1993,[ 9.83821880e-03 -6.55664057e-02 -5.46210706e-...,"{""token_count"":536,""truncated"":false}",,"NOT MY ACCOUNT, NOT AN AUTHORIZED USER {$1800...."
1997,[ 0.03145148 -0.01011822 -0.02316323 -0.025078...,"{""token_count"":123,""truncated"":false}",,After a while the payments became harder and h...
2469,[ 4.74590808e-03 -4.56819348e-02 -2.49751769e-...,"{""token_count"":60,""truncated"":false}",,"In the course of my student loan, I have been ..."
2624,[ 3.91883589e-03 -3.26644145e-02 -7.10378587e-...,"{""token_count"":254,""truncated"":false}",,In accordance with the Fair Credit Reporting A...
2832,[ 8.35181400e-03 -2.91643552e-02 -4.30776961e-...,"{""token_count"":79,""truncated"":false}",,LVNV FUNDING LLC is continually placing a coll...
3328,[ 2.71253809e-02 -1.77491009e-02 -5.32273464e-...,"{""token_count"":156,""truncated"":false}",,On XX/XX/2020 I sent a letter regarding inaccu...
3650,[-6.10093866e-03 -5.93599863e-02 -8.04531425e-...,"{""token_count"":175,""truncated"":false}",,Over a year and a half ago we started the proc...


The model may have encountered errors while calculating embeddings for some rows. Filter out the errored rows before training the model. Alternatively, select these rows and retry the embeddings.

In [11]:
successful_rows = (
    (predicted_embeddings["ml_generate_embedding_status"] == "")
    # Series.str.len() gives the length of an array.
    # See: https://stackoverflow.com/a/41340543/101923
    & (predicted_embeddings["ml_generate_embedding_result"].str.len() != 0)
)
predicted_embeddings = predicted_embeddings[successful_rows]
predicted_embeddings


,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,content
357,[ 1.33585772e-02 -3.76597494e-02 -6.14452176e-...,"{""token_count"":306,""truncated"":false}",,I decided to try XXXX services for my wife and...
428,[ 3.10036819e-04 -3.82593311e-02 -3.41922641e-...,"{""token_count"":134,""truncated"":false}",,XXXX I went to the bank in question ( XXXX XXX...
1319,[ 1.97481886e-02 -1.97448786e-02 -5.13443351e-...,"{""token_count"":215,""truncated"":false}",,I currently have a home loan with my ex husban...
1993,[ 9.83821880e-03 -6.55664057e-02 -5.46210706e-...,"{""token_count"":536,""truncated"":false}",,"NOT MY ACCOUNT, NOT AN AUTHORIZED USER {$1800...."
1997,[ 0.03145148 -0.01011822 -0.02316323 -0.025078...,"{""token_count"":123,""truncated"":false}",,After a while the payments became harder and h...
2469,[ 4.74590808e-03 -4.56819348e-02 -2.49751769e-...,"{""token_count"":60,""truncated"":false}",,"In the course of my student loan, I have been ..."
2624,[ 3.91883589e-03 -3.26644145e-02 -7.10378587e-...,"{""token_count"":254,""truncated"":false}",,In accordance with the Fair Credit Reporting A...
2832,[ 8.35181400e-03 -2.91643552e-02 -4.30776961e-...,"{""token_count"":79,""truncated"":false}",,LVNV FUNDING LLC is continually placing a coll...
3328,[ 2.71253809e-02 -1.77491009e-02 -5.32273464e-...,"{""token_count"":156,""truncated"":false}",,On XX/XX/2020 I sent a letter regarding inaccu...
3650,[-6.10093866e-03 -5.93599863e-02 -8.04531425e-...,"{""token_count"":175,""truncated"":false}",,Over a year and a half ago we started the proc...


We now have the complaints and their text embeddings as two columns in our predicted_embeddings DataFrame.

## Step 2: Create k-means model and predict clusters

In [12]:
from bigframes.ml.cluster import KMeans

cluster_model = KMeans(n_clusters=10) # We will divide our complaints into 10 groups

Perform KMeans clustering

In [13]:
# Use KMeans clustering to calculate our groups. Will take ~3 minutes.
cluster_model.fit(predicted_embeddings[["ml_generate_embedding_result"]])
clustered_result = cluster_model.predict(predicted_embeddings)
# Notice the CENTROID_ID column, which is the ID number of the group that
# each complaint belongs to.
clustered_result.peek(n=5)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,content
1244571,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.414497263076...",[ 1.10590272e-02 -2.11433582e-02 -5.66212423e-...,"{""token_count"":100,""truncated"":false}",,Ive disputed two Bankruptcies that still exist...
744390,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.416584344032...",[ 4.15011719e-02 -4.50705849e-02 -7.35541508e-...,"{""token_count"":100,""truncated"":false}",,The XXXX account was settled as a class action...
127514,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.541137734253...",[ 3.54415141e-02 1.23769706e-02 -2.61783414e-...,"{""token_count"":100,""truncated"":false}",,I have late payments reported on my student lo...
630563,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.477175150810...",[ 2.34235693e-02 -4.21241224e-02 -3.90484147e-...,"{""token_count"":100,""truncated"":false}",,"A Military Star Credit card, aka Take it Home ..."
2651231,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.489760037964...",[ 2.64898203e-02 -5.62610961e-02 -5.82714193e-...,"{""token_count"":101,""truncated"":false}",,My mortgage is with Bank of America. I filed C...


Our DataFrame clustered_result now has an additional column that includes an ID from 1-10 (inclusive) indicating which semantically similar group they belong to.

## Step 3: Use Gemini to summarize complaint clusters

Build prompts - we will choose just two of our categories and prompt GeminiTextGenerator to identify their salient characteristics. The prompt is natural language in a python string.

In [14]:
# Using bigframes, with syntax identical to pandas,
# filter out the first and second groups
cluster_1_result = clustered_result[
    clustered_result["CENTROID_ID"] == 1
][["content"]]
cluster_1_result_pandas = cluster_1_result.head(5).to_pandas()

cluster_2_result = clustered_result[
    clustered_result["CENTROID_ID"] == 2
][["content"]]
cluster_2_result_pandas = cluster_2_result.head(5).to_pandas()

In [15]:
# Build plain-text prompts to send to PaLM 2. Use only 5 complaints from each group.
prompt1 = 'comment list 1:\n'
for i in range(5):
    prompt1 += str(i + 1) + '. ' + \
        cluster_1_result_pandas["content"].iloc[i] + '\n'

prompt2 = 'comment list 2:\n'
for i in range(5):
    prompt2 += str(i + 1) + '. ' + \
        cluster_2_result_pandas["content"].iloc[i] + '\n'

print(prompt1)
print(prompt2)

comment list 1:
1. I currently have a home loan with my ex husband with PHH Mortgages. We filed for divorce and in the divorce decree he became liable for the home and paying the payments. He ended up missing XXXX payments which effected my credit fairly substaintailly. when I became aware of the late payments, I ensured that the account was up to date and have since. I presented to them that I have the legal documents that he is obligated to make the payments each month and that I am not responisble for the payment. I asked them to remove the XXXX dings on my credit and they would not. I offered to present the paperwork and they still would not. The home is now being sold. I even filed with XXXX as a discrepency and they would not remove it. I would have never let these become a late payment. I was not even notified as they had all of his information in the file.
2. In the course of my student loan, I have been making payments and I feel that the payments haven't been added to the deb

In [16]:
# The plain English request we will make of PaLM 2
prompt = (
    "Please highlight the most obvious difference between "
    "the two lists of comments:\n" + prompt1 + prompt2
)
print(prompt)

Please highlight the most obvious difference between the two lists of comments:
comment list 1:
1. I currently have a home loan with my ex husband with PHH Mortgages. We filed for divorce and in the divorce decree he became liable for the home and paying the payments. He ended up missing XXXX payments which effected my credit fairly substaintailly. when I became aware of the late payments, I ensured that the account was up to date and have since. I presented to them that I have the legal documents that he is obligated to make the payments each month and that I am not responisble for the payment. I asked them to remove the XXXX dings on my credit and they would not. I offered to present the paperwork and they still would not. The home is now being sold. I even filed with XXXX as a discrepency and they would not remove it. I would have never let these become a late payment. I was not even notified as they had all of his information in the file.
2. In the course of my student loan, I have

Get a response from Gemini by making a call to Vertex AI using our connection.

In [17]:
from bigframes.ml.llm import GeminiTextGenerator

q_a_model = GeminiTextGenerator()

In [18]:
# Make a DataFrame containing only a single row with our prompt for Gemini
df = bf.DataFrame({"prompt": [prompt]})

In [19]:
# Send the request for Gemini to generate a response to our prompt
major_difference = q_a_model.predict(df)
# PaLM 2's response is the only row in the dataframe result 
major_difference["ml_generate_text_llm_result"].iloc[0]

'The most obvious difference between the two lists of comments is that the first list contains comments about credit report issues related to mortgages and loans, while the second list contains comments about credit report issues related to other types of debts, such as cell phone bills, collections, and bankruptcies.'

We now see GeminiTextGenerator's characterization of the different comment groups. Thanks for using BigQuery DataFrames!

# Summary and next steps

You've used the ML and LLM capabilities of BigQuery DataFrames to help analyze and understand a large dataset of unstructured feedback.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).